Imports

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization , Activation
from tensorflow.keras.regularizers import l2
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam

Load the DB

In [ ]:
(ds_train, ds_val, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train[:80%]', 'train[80%:]', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

num_of_classes = ds_info.features['label'].num_classes
image_shape = ds_info.features['image'].shape
train_size = ds_info.splits['train'].num_examples
test_size = ds_info.splits['test'].num_examples

print(f"Number of Classes: {num_of_classes}")
print(f"Image Shape: {image_shape}")
print(f"Training Set Size: {train_size}")
print(f"Test Set Size: {test_size}")


Number of Classes: 10
Image Shape: (28, 28, 1)
Training Set Size: 60000
Test Set Size: 10000


Data preprocessing

In [ ]:
def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255.0, label

ds_train = ds_train.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache().shuffle(train_size).batch(64).prefetch(tf.data.AUTOTUNE)

ds_val = ds_val.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_val = ds_val.cache().batch(64).prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.cache().batch(64).prefetch(tf.data.AUTOTUNE)

Neuron layers based on my ID

In [ ]:
neurons_layer1 = 38
neurons_layer2 = 91
neurons_layer3 = 90
neurons_layer4 = 5

The Model

In [ ]:
model = Sequential([
    Flatten(input_shape=(28, 28, 1)),

    Dense(neurons_layer1, kernel_regularizer=l2(0.02)),
    Activation(tf.keras.layers.LeakyReLU(alpha=0.01)),
    BatchNormalization(),
    Dropout(0.4),

    Dense(neurons_layer2, kernel_regularizer=l2(0.02)),
    Activation(tf.keras.layers.LeakyReLU(alpha=0.01)),
    BatchNormalization(),
    Dropout(0.4),

    Dense(neurons_layer3, kernel_regularizer=l2(0.02)),
    Activation(tf.keras.layers.LeakyReLU(alpha=0.01)),
    BatchNormalization(),
    Dropout(0.4),

    Dense(neurons_layer4, kernel_regularizer=l2(0.02)),
    Activation(tf.keras.layers.LeakyReLU(alpha=0.01)),
    BatchNormalization(),
    Dropout(0.4),

    Dense(num_of_classes, activation='softmax')
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Summary

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=0.0003),
    loss=SparseCategoricalCrossentropy(),
    metrics=[SparseCategoricalAccuracy()],
)

model.summary()

Model: "sequential_25"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_26 (Flatten)                 │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_126 (Dense)                    │ (None, 38)                  │          29,830 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_101 (Activation)          │ (None, 38)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_100              │ (None, 38)                  │             152 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_100 (Dropout)                │ (None, 38)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_127 (Dense)                    │ (None, 91)                  │           3,549 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_102 (Activation)          │ (None, 91)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_101              │ (None, 91)                  │             364 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_101 (Dropout)                │ (None, 91)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_128 (Dense)                    │ (None, 90)                  │           8,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_103 (Activation)          │ (None, 90)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_102              │ (None, 90)                  │             360 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_102 (Dropout)                │ (None, 90)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_129 (Dense)                    │ (None, 5)                   │             455 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_104 (Activation)          │ (None, 5)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_103              │ (None, 5)                   │              20 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_103 (Dropout)                │ (None, 5)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_130 (Dense)                    │ (None, 10)                  │              60 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 43,070 (168.24 KB)

 Trainable params: 42,622 (166.49 KB)

 Non-trainable params: 448 (1.75 KB)

Training the Model

In [ ]:
model.fit(
    ds_train,
    epochs=25,
    validation_data=ds_val
)

Epoch 1/25
750/750 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - loss: 5.7715 - sparse_categorical_accuracy: 0.2013 - val_loss: 3.1008 - val_sparse_categorical_accuracy: 0.6742
Epoch 2/25
750/750 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - loss: 2.9036 - sparse_categorical_accuracy: 0.4152 - val_loss: 1.5868 - val_sparse_categorical_accuracy: 0.8077
Epoch 3/25
750/750 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - loss: 1.7843 - sparse_categorical_accuracy: 0.5254 - val_loss: 0.9476 - val_sparse_categorical_accuracy: 0.8857
Epoch 4/25
750/750 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 1.3678 - sparse_categorical_accuracy: 0.5875 - val_loss: 0.6898 - val_sparse_categorical_accuracy: 0.9146
Epoch 5/25
750/750 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 1.1921 - sparse_categorical_accuracy: 0.6191 - val_loss: 0.5877 - val_sparse_categorical_accuracy: 0.9134
Epoch 6/25
750/750 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 1.1062 - sparse_categorical_accuracy: 0.6449 - val_loss: 0.4874 - val_sparse_categorical_accuracy: 0.9271
Ep

Model evaluation

In [ ]:
test_loss, test_acc = model.evaluate(ds_val, verbose=2)
print(f"Test accuracy: {test_acc:.4f}")

188/188 - 0s - 2ms/step - loss: 0.3642 - sparse_categorical_accuracy: 0.9410
Test accuracy: 0.9410
